In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
dataset_path = '/content/drive/MyDrive/moviextract'
print(os.listdir(dataset_path))  # List files in the folder

['credits.csv', 'links.csv', 'keywords.csv', 'movies_metadata.csv', 'links_small.csv', 'ratings_small.csv', 'ratings.csv', '.ipynb_checkpoints']


In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [9]:
#### Dataset upload ####
movie = pd.read_csv(dataset_path + '/movies_metadata.csv')

<ipython-input-9-b266747e9bf6>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie = pd.read_csv(dataset_path + '/movies_metadata.csv')


In [4]:
column_names = movie.columns.tolist() # makes the columns into a python list.

# Print the column names
print(column_names)

['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']


In [10]:
movie = movie[['title','overview']].dropna()

In [14]:
movies_sample = movie.head(20000)  # Take only first 20,000 movies
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_sample['overview'])  # Apply TF-IDF on a smaller dataset
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  # Compute similarity

In [15]:
row_count = len(movie)
print("The number of of rows :: ",row_count)

The number of of rows ::  44506


In [16]:
from sklearn.metrics.pairwise import linear_kernel

def recommend_movies(title, movies_sample, cosine_sim):
    # Get the index of the movie in movies_sample
    idx = movies_sample[movies_sample['title'] == title].index

    if len(idx) == 0:
        return "Movie not found in dataset"

    idx = idx[0]

    # Compute similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort movies by similarity score (highest first)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]  # Top 10 recommendations

    # Get the indices of the recommended movies
    movie_indices = [i[0] for i in sim_scores]

    # Return movie titles
    return movies_sample.iloc[movie_indices]['title'].tolist()

# Example usage
print(recommend_movies("The Dark Knight", movies_sample, cosine_sim))


['Foreign Student', "Everybody's All-American", 'Semi-Tough', 'The Express', 'North Dallas Forty', 'Invincible', 'Johnny Be Good', 'All the Right Moves', 'Too Many Girls', 'Side Out']


In [25]:
movie_samples = movie.head(20000)

test_movies = movie_samples['title'].sample(2,random_state = 42).tolist()

print(test_movies)

['The Master of Ballantrae', 'Return to Paradise']


In [27]:
recommend_movie = {}
for movies in test_movies:
  recommend_movie[movies] = recommend_movies(movies, movie_samples, cosine_sim)

print("The prediction based on overview ",recommend_movie)

{'The Master of Ballantrae': ['Thieves', 'Win/Win', "Maria's Lovers", 'ivans xtc.', 'The Swan Princess', 'Ivan the Terrible, Part I', 'Deathstalker II', 'My Friend Ivan Lapshin', 'Yeelen', 'My Little Business'], 'Return to Paradise': ['Postman Blues', 'Macao', 'Kiler', 'The Man with One Red Shoe', "I Just Didn't Do It", 'Nigdy w życiu!', 'The Pilgrim', 'He Ran All The Way', 'The Young Master', 'Chopper']}
